In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Sentences_50Agree.txt to Sentences_50Agree.txt


In [ ]:
from datasets import Dataset
import pandas as pd

# Load file
filepath = "Sentences_AllAgree.txt"

# Parse into text and label
texts, labels = [], []
with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        if "@positive" in line:
            label = "positive"
        elif "@negative" in line:
            label = "negative"
        elif "@neutral" in line:
            label = "neutral"
        else:
            continue  # skip malformed

        text = line.strip().split(f"@{label}")[0].strip()
        texts.append(text)
        labels.append(label)

# Create HF dataset
df = pd.DataFrame({"text": texts, "label": labels})
dataset = Dataset.from_pandas(df)

# Show sample
dataset[0]


{'text': 'According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .',
 'label': 'neutral'}

In [ ]:
def preprocess(example):
    example["input_text"] = f"sentiment analysis: {example['text']}"
    example["target_text"] = example["label"]
    return example

dataset = dataset.map(preprocess)
dataset = dataset.train_test_split(test_size=0.1)

dataset["train"][0]


Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

{'text': 'Profitability ( EBIT % ) was 13.9 % , compared to 13.1 % in the previous-year period .',
 'label': 'positive',
 'input_text': 'sentiment analysis: Profitability ( EBIT % ) was 13.9 % , compared to 13.1 % in the previous-year period .',
 'target_text': 'positive'}

In [ ]:
from transformers import AutoTokenizer

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(example):
    input = tokenizer(example["input_text"], truncation=True, padding="max_length", max_length=128)
    label = tokenizer(example["target_text"], truncation=True, padding="max_length", max_length=8)

    input["labels"] = label["input_ids"]
    return input

tokenized = dataset.map(tokenize, remove_columns=dataset["train"].column_names)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_model, LoraConfig, TaskType

# Load base model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define LoRA config
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none"
)

# Apply LoRA to model
model = get_peft_model(model, peft_config)

# Print trainable params
model.print_trainable_parameters()


trainable params: 344,064 || all params: 77,305,216 || trainable%: 0.4451


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

# Data collator handles padding dynamically
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./lora-finetuned-flan-t5-small",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=50,
    save_steps=200,
    save_total_limit=2,
    num_train_epochs=3,
    learning_rate=5e-4,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=False,
    report_to="none"
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/tmp/ipython-input-7-1651928869.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
50,24.372900
100,7.782900
150,3.931600
200,3.403700
250,3.126600
300,2.907700
350,2.728100
400,2.572600
450,2.461900
500,2.369900


TrainOutput(global_step=765, training_loss=4.409572925442964, metrics={'train_runtime': 109.8018, 'train_samples_per_second': 55.655, 'train_steps_per_second': 6.967, 'total_flos': 285608996241408.0, 'train_loss': 4.409572925442964, 'epoch': 3.0})

In [ ]:
import evaluate
import torch

rouge = evaluate.load("rouge")

def generate_and_evaluate(orig_dataset, num_samples=10):
    model.eval()
    preds = []
    refs = []

    device = next(model.parameters()).device  # get model device (cuda or cpu)

    for example in orig_dataset.select(range(num_samples)):
        inputs = tokenizer(example["input_text"], return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}  # move inputs to model device

        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=10)

        pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
        preds.append(pred)
        refs.append(example["target_text"])

        print(f"Input: {example['input_text']}")
        print(f"Prediction: {pred}")
        print(f"Reference: {example['target_text']}")
        print("-" * 50)

    result = rouge.compute(predictions=preds, references=refs)
    print("ROUGE scores:", result)


generate_and_evaluate(dataset["test"])



Input: sentiment analysis: The power supplies , DC power systems and inverters designed and manufactured by Efore , and systems incorporating them are used in many different applications .
Prediction: neutral
Reference: neutral
--------------------------------------------------
Input: sentiment analysis: The capital structure of Solidium may be complemented by other financial instruments in the future .
Prediction: neutral
Reference: neutral
--------------------------------------------------
Input: sentiment analysis: During the rally , which was authorized by the city administration , a consulate official came out to the workers , spoke to them and took a letter from them .
Prediction: neutral
Reference: neutral
--------------------------------------------------
Input: sentiment analysis: `` After the transaction , Tikkurila has no powder coatings-related operations , '' the company said .
Prediction: neutral
Reference: neutral
--------------------------------------------------
Input:

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

def classify_and_evaluate(orig_dataset, num_samples=100):
    model.eval()
    preds = []
    refs = []

    device = next(model.parameters()).device

    for example in orig_dataset.select(range(num_samples)):
        inputs = tokenizer(example["input_text"], return_tensors="pt", truncation=True, padding="max_length", max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(**inputs, max_length=10)
        pred = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
        preds.append(pred)
        refs.append(example["target_text"].strip().lower())

    # Accuracy
    acc = accuracy_score(refs, preds)

    # Precision, Recall, F1 (macro averaged)
    precision, recall, f1, _ = precision_recall_fscore_support(refs, preds, average='macro')

    print(f"Accuracy: {acc:.4f}")
    print(f"Precision (macro): {precision:.4f}")
    print(f"Recall (macro): {recall:.4f}")
    print(f"F1 (macro): {f1:.4f}")
    print("\nClassification Report:")
    print(classification_report(refs, preds))

classify_and_evaluate(dataset["test"])


Accuracy: 0.9200
Precision (macro): 0.9283
Recall (macro): 0.8839
F1 (macro): 0.9015

Classification Report:
              precision    recall  f1-score   support

    negative       0.93      0.93      0.93        14
     neutral       0.91      0.98      0.95        63
    positive       0.94      0.74      0.83        23

    accuracy                           0.92       100
   macro avg       0.93      0.88      0.90       100
weighted avg       0.92      0.92      0.92       100

